## **Experiment 1: Hybrid/Ensemble Retriever**

Combine sparse retriever (BM25 that is based on keyword search) with dense retriever (FAISS that is based on embedding similarity/semantic similarity).

Results are reranked based on the Reciprocal Rank Fusion algorithm (RRF).

The Hybrid/Ensemble approach, combining basic keyword and similarity search, could be applicable to our use case to retrieve a interpolation of keyword-wise relevant (BM25) and semantically-similar (FAISS) results.

In [ ]:
%pip install --quiet --upgrade bitsandbytes langchain langchain-community langchain-huggingface transformers beautifulsoup4 faiss-gpu rank_bm25 lark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.3 MB/s eta 0:00:00


In [ ]:
from langchain_core.documents import Document
from langchain.retrievers import EnsembleRetriever # Supports Ensembling of results from multiple retrievers
from langchain_community.retrievers import BM25Retriever
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
import nltk
from nltk.corpus import stopwords
import re

**Setup Simple Experiment Data for Experiment 1**

We first set up simple experiment data where we have 2 short documents for each activity category. This is done to have an easier <u>preliminary understanding</u> of the behaviour of the individual retrievers as we are able to quickly look at what words our data contains.

In [ ]:
# Simple experiment example data
docs = [
    Document(
        page_content="The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.",
        metadata={"activity": 'Hiking', "country": 'Norway'},
    ),
    Document(
        page_content="Unique hike that can be done are volcanic hikes which can be done in Iceland. It is recommended to go with a tour of experienced people!",
        metadata={"activity": 'Hiking', "country": 'Iceland'},
    ),
    Document(
        page_content="Popular food in Norway is seafood! The best seafood in the Nordic region can be found in Norway. The seafood is freshly caught from the arctic ocean. Popular choices include the famous norwegian salmon. Other delicacies include whale steak!",
        metadata={"activity": 'Food', "country": 'Norway'},
    ),
    Document(
        page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger.",
        metadata={"activity": 'Food', "country": 'Norway'},
    ),
    Document(
        page_content="Transportation within Reykjavik is fairly convenient as there is a public bus service called BSI. All you need to do is to download their mobile app, follow the instructions, and you're good to go. Transportation to places outside Reykjavik however requires a car. Some options include car rentals as well as booking bus tours.",
        metadata={"activity": 'Transportation', "country": 'Iceland'},
    ),
    Document(
        page_content="Finland is easily accessible with its HSL public transportation services where all you need to do is to download a mobile app and follow the instructions.",
        metadata={"activity": 'Transportation', "country": 'Iceland'},
    ),
    Document(
        page_content="Finland is known for its snowy-like landscape and captivating auroras. One of the best places to stay is the Glass huts in Skyfire village in Rovaniemi, Lapland where you can admire the beautiful northern lights and snowy landscape. The village has its very own restaurant called Sky Huts Restaurant and Bar which offers tailor-made menus by a professional chef using local ingredients.",
        metadata={"activity": 'Accomodation', "country": 'Finland'},
    ),
    Document(
        page_content="A nice place to stay in Norway is the Lofoten Islands, in particlar Unstad which provides a breathtaking view of the mountain valley, ocean, and if you're lucky, northern lights.",
        metadata={"activity": 'Accomodation', "country": 'Norway'},
    ),
]

In [ ]:
# Initialise BM25 retreiver (keyword-based)
bm25_retriever = BM25Retriever.from_documents(
    docs
)
bm25_retriever.k = 5

# Initialise the FAISS retriever (semantic-similarity)
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world")))
faiss_vector_store = FAISS(
    embedding_function=embeddings_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

faiss_vector_store.add_documents(docs)
faiss_retriever = faiss_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5}) # num docs to return from FAISS

# Initialise the hybrid/ensemble retriever
# Uses RRF to sum the rankings of each doc from both retrievers, discounting rankings that are lower.
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

**Test Query 1: Testing Hybrid/Ensemble Retriever**

In [ ]:
test_query_1 = "What are the best hikes?"

In [ ]:
docs = ensemble_retriever.invoke(test_query_1)
docs

[Document(metadata={'activity': 'Hiking', 'country': 'Norway'}, page_content='The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.'),
 Document(metadata={'activity': 'Hiking', 'country': 'Iceland'}, page_content='Unique hike that can be done are volcanic hikes which can be done in Iceland. It is recommended to go with a tour of experienced people!'),
 Document(metadata={'activity': 'Accomodation', 'country': 'Finland'}, page_content='Finland is known for its snowy-like landscape and captivating auroras. One of the best places to stay is the Glass huts in Skyfire village in Rovaniemi, Lapland where you can admire the beautiful northern lights and snowy landscape. The village has its v

**Test Query 2: Testing Hybrid/Ensemble Retriever**

In [ ]:
test_query_2 = "What are the best food?"

In [ ]:
docs = ensemble_retriever.invoke(test_query_2)
docs

[Document(metadata={'activity': 'Hiking', 'country': 'Norway'}, page_content='The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.'),
 Document(metadata={'activity': 'Food', 'country': 'Norway'}, page_content='Popular food in Norway is seafood! The best seafood in the Nordic region can be found in Norway. The seafood is freshly caught from the arctic ocean. Popular choices include the famous norwegian salmon. Other delicacies include whale steak!'),
 Document(metadata={'activity': 'Accomodation', 'country': 'Finland'}, page_content='Finland is known for its snowy-like landscape and captivating auroras. One of the best places to stay is the Glass huts in Skyfire village in Rovaniemi, 

**Questions about Hybrid/Ensemble Experiment**
- For <u>Test Query 2</u>, why is a hiking related document being returned first when we are making a query about food?

**Investigating BM25**

<u>Running BM25 with Test Query 2</u>

In [ ]:
bm25_retriever = BM25Retriever.from_documents(
    docs
)
bm25_retriever.k = 5
bm25_retriever.invoke(test_query_2)

[Document(metadata={'activity': 'Hiking', 'country': 'Norway'}, page_content='The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail can be quite demanding as the steps are quite steep.'),
 Document(metadata={'activity': 'Hiking', 'country': 'Iceland'}, page_content='Unique hike that can be done are volcanic hikes which can be done in Iceland. It is recommended to go with a tour of experienced people!'),
 Document(metadata={'activity': 'Food', 'country': 'Norway'}, page_content='Popular food in Norway is seafood! The best seafood in the Nordic region can be found in Norway. The seafood is freshly caught from the arctic ocean. Popular choices include the famous norwegian salmon. Other delicacies include whale steak!'),
 Document(metadata=

<u>Running BM25 with the keyword that we want: 'food'</u>

In [ ]:
bm25_retriever = BM25Retriever.from_documents(
    docs
)
bm25_retriever.k = 5
bm25_retriever.invoke("food")

[Document(metadata={'activity': 'Food', 'country': 'Norway'}, page_content='Popular food in Norway is seafood! The best seafood in the Nordic region can be found in Norway. The seafood is freshly caught from the arctic ocean. Popular choices include the famous norwegian salmon. Other delicacies include whale steak!'),
 Document(metadata={'activity': 'Food', 'country': 'Norway'}, page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger."),
 Document(metadata={'activity': 'Hiking', 'country': 'Iceland'}, page_content='Unique hike that can be done are volcanic hikes which can be done in Iceland. It is recommended to go with a tour of experienced people!'),
 Document(metadata={'activity': 'Accomodation', 'country': 'Norway'}, page_content="A nice place to stay in Norway is the Lofoten Islands, in particlar Unstad which p

<u>Findings from BM25</u>

When using BM25 with the full query, why were the hiking-related documents ranked higher than the food-related documents?
- Other terms in the query are also considered (because stopwords are not removed) into the the BM25 calculation. For instance, the word 'best' and 'are' which are present in the hiking-related documents.


<u>How can we improve BM25</u>
- Drop stopwords from the query so that they are not factored into the BM25 calculation

In [ ]:
nltk.download('stopwords')
print("The stopwords provided by the nltk library include:")
print(stopwords.words('english'))
nltk_stopwords = stopwords.words('english')

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
full_query = test_query_2
query_words = re.findall(r'\b\w+\b', full_query)
keywords = [keyword for keyword in query_words if keyword.lower() not in nltk_stopwords]
full_query_keywords_only = ' '.join(keywords)
print(f'The original query is: "{full_query}"\n')
print(f'The new query is after removing stopwords is: "{full_query_keywords_only}"\n')

bm25_retriever = BM25Retriever.from_documents(
    docs
)
bm25_retriever.k = 5
bm25_retriever.invoke(full_query_keywords_only)

The original query is What are the best food?

The new query is best food



[Document(metadata={'activity': 'Food', 'country': 'Norway'}, page_content='Popular food in Norway is seafood! The best seafood in the Nordic region can be found in Norway. The seafood is freshly caught from the arctic ocean. Popular choices include the famous norwegian salmon. Other delicacies include whale steak!'),
 Document(metadata={'activity': 'Food', 'country': 'Norway'}, page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger."),
 Document(metadata={'activity': 'Hiking', 'country': 'Iceland'}, page_content='Unique hike that can be done are volcanic hikes which can be done in Iceland. It is recommended to go with a tour of experienced people!'),
 Document(metadata={'activity': 'Accomodation', 'country': 'Norway'}, page_content="A nice place to stay in Norway is the Lofoten Islands, in particlar Unstad which p

**We can see an improvement in the BM25 ranking by using removing stopwords from the query**

**Investigating FAISS**

<u>Running FAISS (IndexL2) with the Test Query 2</u>

In [ ]:
faiss_retriever.invoke(test_query_2)

[Document(metadata={'activity': 'Food', 'country': 'Norway'}, page_content='Popular food in Norway is seafood! The best seafood in the Nordic region can be found in Norway. The seafood is freshly caught from the arctic ocean. Popular choices include the famous norwegian salmon. Other delicacies include whale steak!'),
 Document(metadata={'activity': 'Food', 'country': 'Norway'}, page_content="The famous street food of Iceland is the Hotdog! It is called the Baejarins Beztu Pylsur hot dog is made of a mix of lamb, beef and pork. Other delicacies of iceland include Fish and Chips as well as Tommi's burger."),
 Document(metadata={'activity': 'Hiking', 'country': 'Norway'}, page_content='The best hikes in Norway include the Reinebringen hike in the Lofoten islands. At a modest 448 meters high, Reinebringen is far from one of the highest peaks on the Lofoten islands. Yet this is more than made up for by the iconic view from the summit of Reine. It is not suitable for winter! Also, the trail

It can be observed that the embedding similarity works well even though the words in the query are not an exact match. This is because of the vector similarity between words in the user query and document.

**TODO: Use the full data and a number of queries to test how the different weightages in Hybrid/Ensemble Retriever affect retrieval performance**
- How to measure performance?

**Conclusion for Experiment 1: Hybrid/Ensemble Retriever**

We can improve Hybrid/Ensemble Retriever by removing stopwords to increase the performance of BM25.